In [561]:
import numpy as np

In [925]:
# total capacity of the queue
Qmax = 5

# allowed actions an associated costs
actions = [0,    1,    2,    3]
C =       [0, -0.5, -2.5, -7.0]

# holding costs
Chold = [0, -1, -2, -5, -7, -10]

# distribution of arrivals (make sure to specify for 
# all [0, ..., Qmax] to avoid problems with np.put())
A = np.array([0.5, 0.0, 0.4, 0, 0, 0.1])

# construct the transition matrix for action 0...
p0 = np.zeros((Qmax + 1, Qmax + 1))
for i in range(Qmax+1):
    indices = range(i, Qmax)
    values = A[0:Qmax - i]
    np.put(p0[i], indices, values)

    # last entry is probability of reaching full capacity
    p0[i, Qmax] = sum(A[Qmax - i:Qmax+1])

# ...from which transitions for other actions are simply derived
def p(a, i, j):
    return p0[max(i - a, 0), j]

# rewards
def r(a, i):
    return Chold[i] + C[a]

In [926]:
# default policy (always take action 1)
policy0 = np.ones((Qmax + 1), dtype=np.int16)

In [963]:
def apply_policy(pol):
    # compute rewards induced by policy
    rf = np.zeros((Qmax + 1, 1))
    for i in range(Qmax + 1):
        rf[i, 0] = r(pol[i], i)
    
    # compute transitions induced by policy
    pf = np.zeros((Qmax + 1, Qmax + 1))
    for i in range(Qmax + 1):
        for j in range(Qmax + 1):
            pf[i, j] = p(pol[i], i, j)
    
    return rf, pf

# evaluate policy using average reward criterium by solving (2.16)
def evaluate_avg(pol):
    rf, pf = apply_policy(pol)
    
    # solve system (2.16) using least squares
    # (because the system is rank-deficient)
    a = np.hstack([pf - np.eye(Qmax + 1), -np.ones((Qmax + 1, 1))])
    b = - rf
    x, _, _, _ = np.linalg.lstsq(a, b, rcond=None)
    d = x[0:Qmax+1, 0]
    g = x[-1, 0]
    return d, g

# evaluate policy using discounted total reward criterium
# by solving (3.3)
def evaluate_disc(pol, alpha=0.95):
    rf, pf = apply_policy(pol)

    a = alpha * pf - np.eye(Qmax + 1)
    b = - rf
    v = np.linalg.solve(a, b)
    return v


In [964]:
d0, g0 = evaluate_avg(policy0)
print(g0)

-6.480113636363644


In [965]:
v0 = evaluate_disc(policy0, 0.95)
print(v0)

[[-111.34369133]
 [-112.34369133]
 [-117.08868158]
 [-125.69391909]
 [-134.5133063 ]
 [-141.70251522]]


In [966]:
# verify avg and disc criteria by taking alpha close to 1
alpha = 0.99999
res = evaluate_disc(policy0, alpha) * (1 - alpha)
print(res)

[[-6.4799079 ]
 [-6.4799179 ]
 [-6.47997181]
 [-6.48006908]
 [-6.48016987]
 [-6.48025027]]


In [967]:
# just using plain Python here, but we could have
# used matrix multiplication for the second term
def reward(v, i, a, alpha):
    return r(a, i) + alpha * sum([
        p(a, i, j) * v[j]
        for j in range(Qmax + 1)
    ])


# apply one step of policy improvement
# v is value or relative rewards
# make sure to use alpha=1 in average reward case
def policy_improv(policy, v, alpha=1):
    new_policy = np.copy(policy)

    # for all states, compute the optimal action
    for i in range(Qmax + 1):
        # current reward
        best_reward = reward(v, i, policy[i], alpha)
        for a in actions:
            if a == policy[i]:
                continue
        
            current = reward(v, i, a, alpha)
            if current > best_reward:
                best_reward = current
                new_policy[i] = a

    return new_policy

In [961]:
policy_improv(policy0, v0, alpha=0.95)

array([0, 1, 2, 2, 3, 3], dtype=int16)

In [962]:
policy_improv(policy0, d0)

array([0, 1, 2, 2, 3, 3], dtype=int16)

In [968]:
# average reward policy iteration
policy = np.ones((Qmax + 1), dtype=np.int16)
prev = None
while not np.array_equal(policy, prev):
    prev = np.copy(policy)
    d, g = evaluate_avg(policy)
    print(policy)
    print(g)
    policy = policy_improv(policy, d)

print(policy)

[1 1 1 1 1 1]
-6.480113636363644
[0 1 2 2 3 3]
-4.2349999999999985
[0 1 2 2 2 3]
-4.208333333333335
[0 1 2 2 2 3]


In [971]:
# successive approximation
def succ_approx(v0, epsilon=0.001):
    v = np.copy(v0) + 2 * epsilon * np.ones_like(v0)
    diff = 2 * epsilon
    while diff > epsilon:
        vprev = np.copy(v)
        for i in range(Qmax + 1):
            v[i] = max([reward(vprev, i, a, 1) for a in actions])
            
        M = np.max(np.abs(v - vprev))
        m = np.min(np.abs(v - vprev))
        diff = M - m
    
    # determine policy
    policy = np.zeros_like(v0)
    for i in range(Qmax + 1):
        policy[i] = np.argmax([reward(v, i, a, 1) for a in actions])
    
    return v, policy

In [973]:
succ_approx(np.zeros((Qmax + 1)))

(array([-42.74885045, -44.24885045, -47.24885045, -53.49864598,
        -59.33174775, -66.83174775]),
 array([0., 1., 2., 2., 2., 3.]))